In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [59]:
class myCallback (tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95 and logs.get('val_accuracy')>0.95):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
def create_model():
    mobilenet_v3 = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/small-100-224-feature-vector/versions/1"
    mobile_net_layers = hub.KerasLayer(mobilenet_v3, input_shape=(224,224,3))
    mobile_net_layers.trainable = False

    model = tf.keras.Sequential([
        mobile_net_layers,
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(22,activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data/training/'
    validation_dir = '../data/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=callback
    )
    
    return model

In [60]:
def create_model():
    mobilenet_v3 = tf.keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top=False)


    model = tf.keras.Sequential([
        mobilenet_v3,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(10,activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data3/training/'
    validation_dir = '../data3/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    model.fit(
        train_generator,
        epochs=300,
        validation_data=validation_generator,
        callbacks=callback
    )
    
    return model

In [62]:
model = create_model()
model.save("./model.v1-5.h5")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 global_average_pooling2d_4   (None, 576)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 576)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5770      
                                                                 
Total params: 944,890
Trainable params: 932,778
Non-trainable params: 12,112
_________________________________________________________________
Found 398 images belonging to 10 classes.
F

In [58]:
# Specify the local image path
import numpy as np
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

local_image_path = '../data3/validation/Ayam Goreng/20210605215357-1-ilustrasi-ayam-sasando-001-tantri-setyorini.jpg'

# Make predictions for the local image
img = image.load_img(local_image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)
img_array = img_array/255.

# Make predictions
train_dir = "../data3/training/"
classes = [f for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))]
predictions = model.predict(img_array)
print(predictions * 100)

predicted_class = np.argmax(predictions)
predicted_food = classes[predicted_class]

print(f"The predicted class is: {predicted_class}")
print(f"The predicted food is: {predicted_food}")

1/1 [==============================] - 0s 30ms/step
[[4.8728272e-01 9.9262718e+01 8.1784159e-02 1.3537657e-01 1.6016125e-03
  4.8219943e-03 1.3671792e-03 8.3298981e-04 7.7417870e-03 1.6467793e-02]]
The predicted class is: 1
The predicted food is: Ayam Goreng
